In [100]:
from gurobipy import *
from pulp import *

In [240]:
# Reading instance
'''
    read_instance(file_name: str)
    read a instance file and return it values
    :return: (n, m, graph, req), where
    n: int - number of vertices
    m: int - number of edges
    graph: [[int]] - an adjacency matrix
    req: [[float]] - an matrix of requirements (sup trian)
'''
def read_instance(file_name: str) -> (int, int, [[int]], [[float]]):
    with open(file_name, 'r') as instance_file:
        (n, m) = map(lambda x: int(x), instance_file.readline().split())
        graph = [[ -1 for _ in range(n)] for _ in range(n)]
        req = [[ 0.0 for _ in range(n)] for _ in range(n)]
        for _ in range(m):
            # length(u,v) = l
            (u, v, l) = map(lambda x: int(x), instance_file.readline().split())
            graph[u][v] = l
            graph[v][u] = l
        # main diagonal equals 0
        for i in range(n):
            graph[i][i] = -1
        for i in range(n):
            for j in range(i,n):
                if j == i:
                    req[i][i] = 0.0
                else:
                    r = float(instance_file.readline())
                    req[i][j] = r
                    req[j][i] = r
        
    return (n, m, graph, req)

In [241]:
'''
    get_o_u(req)
    Generate O value for each vertice u
    return: o, an [float]
'''
def get_o_u(req) -> [[float]]:
    o = []
    for u in range(n):
        o.append(sum(req[u][v] for v in range(u,n)))
    return o

In [274]:
'''
    defining_vars(graph: [[int]])
    generate MIP vars
    return: (x, y, f), where:
    x: dict(LPVar): binary x[i][j]
    y: dict(LPVar): binary y[u][i][j]
    f: dict(LPVar): cont   f[u][i][j]
'''
def defining_vars(graph: [[int]], n: int) -> (dict, dict, dict):
    x = LpVariable.dicts("x", [(i,j) for i in range(n) for j in range(i, n) if graph[i][j] >= 0],
                        lowBound=0, upBound=1, cat='Integer')
    y = LpVariable.dicts("y", [(u,i,j) for u in range(n) for i in range(n) for j in range(n) if graph[i][j] >= 0],
                        lowBound=0, upBound=1, cat='Integer')
    f = LpVariable.dicts("f", [(u,i,j) for u in range(n) for i in range(n) for j in range(n) if graph[i][j] >= 0],
                        lowBound=0)
    return (x, y, f)

In [275]:
'''
    defining_objective(graph: [[int]], f: dict)
    Generate objective function
    return: problem, an LpProblem
'''
def defining_objective(graph: [[int]], n: int, f: dict):
    problem = LpProblem("OCT problem", LpMinimize)
    problem += lpSum(graph[i][j] * f[u, i, j] for i in range(n) for j in range(i,n) for u in range(n) if graph[i][j] >= 0), "communication cost"
    return problem

In [276]:
def first_constraint(n, o, problem, f):
    for u in range(n):
        problem += lpSum(f[u, u, j] for j in range(n) if j != u) == o[u]
    return problem

In [277]:
def second_constraint(n, req, problem, f):
    for u in range(n):
        for i in range(n):
            problem += lpSum(f[u, i, j] - f[u, j, i] for j in range(n) if (j != u) and (j != i)) == - req[u][i]
    return problem

In [278]:
def third_fourth_constraint(n, o, problem, f, y):
    for u in range(n):
        for i in range(n):
            for j in range(i, n):
                if i==j: continue
                problem += f[u,i,j] <= o[u] * y[u,i,j]
                problem += f[u,j,i] <= o[u] * y[u,j,i]
    return problem

In [279]:
def fifth_constraint(n, problem, y):
    for u in range(n):
        problem += lpSum(y[u, i, j] for i in range(n) for j in range(i, n) if (j != u) and (j != i)) == n-1
    return problem

In [280]:
def sixth_constraint(n, problem, x, y):
    for u in range(n):
        for i in range(n):
            for j in range(i, n):
                if i==j: continue
                problem += y[u,i,j] + y[u,j,i] <= x[i,j]
    return problem

In [281]:
def seventh_constraint(n, problem, x):
    problem += lpSum(x[i,j] for i in range(n) for j in range(i,n) if i != j) == n-1
    return problem

In [282]:
(n, m, graph, req) = read_instance('./instances/well-known/berry6.in')
o = get_o_u(req)

In [283]:
(x, y, f) = defining_vars(graph, n)

In [284]:
problem = defining_objective(graph, n, f)
problem = first_constraint(n, o, problem, f)
problem = second_constraint(n, req, problem, f)
problem = third_fourth_constraint(n, o, problem, f, y)
problem = fifth_constraint(n, problem, y)
problem = sixth_constraint(n, problem, x, y)
problem = seventh_constraint(n, problem, x)

In [285]:
solver = GUROBI(timeLimit = 3600)
solver.buildSolverModel(problem)
solver.callSolver(problem)
solver.findSolutionValues(problem)

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 319 rows, 375 columns and 1020 nonzeros
Model fingerprint: 0xc1ca9506
Variable types: 180 continuous, 195 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+01]
Presolve removed 35 rows and 50 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi status= 3


<ipython-input-285-13dbcef67e8e>:2: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  solver.buildSolverModel(problem)


-1